## Object Detection from Webcam Stream using Edge TPU and SSD MobileNet V2
This notebook performs object detection on a webcam stream using the `ssd_mobilenet_v2_coco_quant_postprocess_edgetpu.tflite` model (available from the Coral website [here](https://coral.ai/models/object-detection/)). Object detection itself is done on a connected Coral USB accelerator.

In [1]:
import tensorflow as tf
import platform
import cv2
import numpy as np
import platform
import collections
import tflite_runtime.interpreter as tflite
from pycoral.adapters import detect
from pycoral.adapters import common

In [2]:
EDGETPU_SHARED_LIB = {
  'Linux': 'libedgetpu.so.1',
  'Darwin': 'libedgetpu.1.dylib',
  'Windows': 'edgetpu.dll'
}[platform.system()]

In [3]:
TFLITE_FILE_PATH = r"models_1\ssd_mobilenet_v2_coco_quant_postprocess_edgetpu.tflite"

interpreter = tf.lite.Interpreter(TFLITE_FILE_PATH, experimental_delegates=[tflite.load_delegate(EDGETPU_SHARED_LIB)])

In [4]:
#Label loading function taken from tflite GitHub repo: https://github.com/google-coral/tflite/tree/master/python/examples/classification
def load_labels(path, encoding='utf-8'):
  """Loads labels from file (with or without index numbers).
  Args:
    path: path to label file.
    encoding: label file encoding.
  Returns:
    Dictionary mapping indices to labels.
  """
  with open(path, 'r', encoding=encoding) as f:
    lines = f.readlines()
    if not lines:
      return {}

    if lines[0].split(' ', maxsplit=1)[0].isdigit():
      pairs = [line.split(' ', maxsplit=1) for line in lines]
      return {int(index): label.strip() for index, label in pairs}
    else:
      return {index: line.strip() for index, line in enumerate(lines)}

Class = collections.namedtuple('Class', ['id', 'score'])

In [5]:
LABEL_PATH = r"models_1\coco_labels.txt"
labels = load_labels(LABEL_PATH)

In [6]:
capture = cv2.VideoCapture(0)

In [7]:
interpreter.allocate_tensors()

while True:

    ret, image_np = capture.read()
    
    image_np = cv2.resize(image_np, (300, 300))
    
    common.set_input(interpreter, image_np)
    
    interpreter.invoke()
    
    objs = detect.get_objects(interpreter, score_threshold = 0.5)
    
    for obj in objs:
        cv2.rectangle(image_np, (obj.bbox.xmin, obj.bbox.ymin), (obj.bbox.xmax, obj.bbox.ymax), (0, 255, 0), 2)
        cv2.putText(image_np, labels.get(obj.id, obj.id) + ' ' + str(round(obj.score, 2)), (obj.bbox.xmin, obj.bbox.ymin - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255)
    
    cv2.imshow('Object Detection', image_np)
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

### Results

<table><tr>
<td> <img src='person_recognized_1.PNG' style="width: 250px;"/> </td>
<td> <img src='objects_recognized_1.PNG' style="width: 250px;"/> </td>
</tr></table>